In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import re

In [2]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 19:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transaction_20210228_20210827_sdf = spark.read.parquet("../data/curated/transactions_20210228_20210827_all_details")
transaction_20210828_20220227_sdf = spark.read.parquet("../data/curated/transactions_20210828_20220227_all_details")
transaction_20220228_20220828_sdf = spark.read.parquet("../data/curated/transactions_20220228_20220828_all_details")
transaction_20210228_20210827_sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20,Ultricies Digniss...,"gift, card, novel...",b,4.6,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20,Egestas Blandit Ltd,tent and awning s...,b,3.1,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20,Nullam Enim Ltd,"computers, comput...",b,3.2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,17324645993,13.474545944752698,4ac6ef24-80dd-4c2...,2021-08-19,Eget Metus In Cor...,tent and awning s...,a,5.7,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,96161808980,135.71010371116802,ae78d6ce-a28e-4b4...,2021-08-22,Velit Aliquam Cor...,digital goods: bo...,a,7.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,83620670827,195.78268682763712,14622aee-0f3b-488...,2021-08-15,Curabitur Massa C...,"cable, satellite,...",b,3.4,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,32234779638,44.74559545873514,d3f8066f-2a0b-4c1...,2021-07-15,null,null,null,null,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,28935822912,17.545127019931726,6c7115ea-1834-40c...,2021-08-27,Conubia Nostra Li...,shoe shops,c,2.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,68216911708,35.93036528757969,3ff61b95-9e5d-491...,2021-08-13,Placerat Eget Ven...,"computers, comput...",c,3.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
3,55090181035,269.6242486472489,8710d587-ada5-496...,2021-07-16,Dictum Ultricies Ltd,computer programm...,a,6.4,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530


# Obtaining month from dates

In [4]:
transaction_20210228_20210827_sdf = transaction_20210228_20210827_sdf.withColumn("month", F.date_format('order_datetime', 'MM'))
transaction_20210828_20220227_sdf = transaction_20210828_20220227_sdf.withColumn("month", F.date_format('order_datetime', 'MM'))
transaction_20220228_20220828_sdf = transaction_20220228_20220828_sdf.withColumn("month", F.date_format('order_datetime', 'MM'))
transaction_20210228_20210827_sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id,month
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20,Ultricies Digniss...,"gift, card, novel...",b,4.6,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20,Egestas Blandit Ltd,tent and awning s...,b,3.1,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20,Nullam Enim Ltd,"computers, comput...",b,3.2,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,17324645993,13.474545944752698,4ac6ef24-80dd-4c2...,2021-08-19,Eget Metus In Cor...,tent and awning s...,a,5.7,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,96161808980,135.71010371116802,ae78d6ce-a28e-4b4...,2021-08-22,Velit Aliquam Cor...,digital goods: bo...,a,7.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,83620670827,195.78268682763712,14622aee-0f3b-488...,2021-08-15,Curabitur Massa C...,"cable, satellite,...",b,3.4,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,32234779638,44.74559545873514,d3f8066f-2a0b-4c1...,2021-07-15,null,null,null,null,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,07
3,28935822912,17.545127019931726,6c7115ea-1834-40c...,2021-08-27,Conubia Nostra Li...,shoe shops,c,2.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,68216911708,35.93036528757969,3ff61b95-9e5d-491...,2021-08-13,Placerat Eget Ven...,"computers, comput...",c,3.0,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,08
3,55090181035,269.6242486472489,8710d587-ada5-496...,2021-07-16,Dictum Ultricies Ltd,computer programm...,a,6.4,Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530,07


# Aggregation for all released transactions
collecting by each merchant:
- monthly revenue
- monthly revenue standard deviation
- monthly order numbers
- monthly distinct customers
- monthly take portion

In [49]:
# using count() to convert back to dataframe, instead of a grouped object
# groupby merchant and month.
# get monthly revenue, monthly order numbers, monthly distinct customers

count_transaction_20210228_20210827_sdf = transaction_20210228_20210827_sdf \
                                                .groupBy("merchant_abn", "month") \
                                                .agg(F.countDistinct("consumer_id"),
                                                     F.count("merchant_abn"),
                                                     F.sum("dollar_value")
                                                    )
aggregate_transaction_20210228_20210827_sdf = transaction_20210228_20210827_sdf \
                                                .join(count_transaction_20210228_20210827_sdf, 
                                                       ["merchant_abn", "month"]) \
                                                .drop_duplicates()
# get monthly take portion
aggregate_transaction_20210228_20210827_sdf = aggregate_transaction_20210228_20210827_sdf \
                                                .withColumn("take_rate",
                                                F.col("take_rate").cast('double'))
aggregate_transaction_20210228_20210827_sdf = aggregate_transaction_20210228_20210827_sdf \
                                                .withColumn("take_portion",
                                                (F.col("take_rate") / 100) * (F.col("sum(dollar_value)")))

aggregate_transaction_20210228_20210827_sdf = aggregate_transaction_20210228_20210827_sdf\
                                                 .withColumnRenamed("count(consumer_id)", "distinct_monthly_consumer_count") \
                                                 .withColumnRenamed("count(merchant_abn)", "monthly_order_count") \
                                                 .withColumnRenamed("sum(dollar_value)", "monthly_revenue")

# quick check
aggregate_transaction_20210228_20210827_sdf.where(F.col("merchant_abn") == 46012371285).orderBy("month")


merchant_abn,month,user_id,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id,distinct_monthly_consumer_count,monthly_order_count,monthly_revenue,take_portion
46012371285,02,8630,520.0048999285881,944f89fd-e0bb-4af...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Sara Jackson,5218 Chase Walk S...,QLD,4824,Female,436263,15,15,8468.252368960671,406.47611371011226
46012371285,02,2860,704.6535251293501,6342d431-e4c8-4cd...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Patricia Francis,963 Davis Falls,WA,6832,Female,88737,15,15,8468.252368960671,406.47611371011226
46012371285,02,10468,309.28206440311914,a1c37e57-5d4e-4a2...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Vicki Pitts,5174 Moss Crossin...,VIC,3269,Female,401208,15,15,8468.252368960671,406.47611371011226
46012371285,02,2596,1539.6360111897984,09a28589-c3b3-4df...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Ashley Gardner,416 Tonya Passage,NSW,2075,Female,547672,15,15,8468.252368960671,406.47611371011226
46012371285,02,23968,601.0471256457166,4c278e0c-33ff-417...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,James Gonzales,88766 Juarez Lock...,TAS,7268,Male,1123368,15,15,8468.252368960671,406.47611371011226
46012371285,02,3830,614.7830869551526,6273e424-fe9f-4e7...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Cheryl Huang MD,9280 Jessica Turn...,VIC,3005,Female,1433069,15,15,8468.252368960671,406.47611371011226
46012371285,02,19199,507.1868370194734,d68949e3-feec-478...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Jennifer Newton,5543 Kelli Mounta...,NSW,2082,Female,704195,15,15,8468.252368960671,406.47611371011226
46012371285,02,374,337.87155609626785,8aa70834-07b7-40d...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Erica Wood,766 Patterson Cap...,WA,6361,Female,186883,15,15,8468.252368960671,406.47611371011226
46012371285,02,21291,495.39247930245097,79077155-7af5-463...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Elaine Collins,160 Glenn Forks A...,WA,6740,Female,1480599,15,15,8468.252368960671,406.47611371011226
46012371285,02,17588,303.23393013006955,6c0a083b-63a6-4ee...,2021-02-28,A Ultricies Inc.,"books, periodical...",b,4.8,Linda Terry,925 Shaun Motorwa...,VIC,3161,Female,482126,15,15,8468.252368960671,406.47611371011226


In [51]:
# using count() to convert back to dataframe, instead of a grouped object
# groupby merchant and month.
# get monthly revenue, monthly order numbers, monthly distinct customers

count_transaction_20210828_20220227_sdf = transaction_20210828_20220227_sdf \
                                                .groupBy("merchant_abn", "month") \
                                                .agg(F.countDistinct("consumer_id"),
                                                     F.count("merchant_abn"),
                                                     F.sum("dollar_value")
                                                    )
aggregate_transaction_20210828_20220227_sdf = transaction_20210828_20220227_sdf \
                                                .join(count_transaction_20210828_20220227_sdf, 
                                                       ["merchant_abn", "month"]) \
                                                .drop_duplicates()
# get monthly take portion
aggregate_transaction_20210828_20220227_sdf = aggregate_transaction_20210828_20220227_sdf \
                                                .withColumn("take_rate",
                                                F.col("take_rate").cast('double'))
aggregate_transaction_20210828_20220227_sdf = aggregate_transaction_20210828_20220227_sdf \
                                                .withColumn("take_portion",
                                                (F.col("take_rate") / 100) * (F.col("sum(dollar_value)")))

aggregate_transaction_20210828_20220227_sdf = aggregate_transaction_20210828_20220227_sdf\
                                                 .withColumnRenamed("count(consumer_id)", "distinct_monthly_consumer_count") \
                                                 .withColumnRenamed("count(merchant_abn)", "monthly_order_count") \
                                                 .withColumnRenamed("sum(dollar_value)", "monthly_revenue")
aggregate_transaction_20210828_20220227_sdf

merchant_abn,month,user_id,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id,distinct_monthly_consumer_count,monthly_order_count,monthly_revenue,take_portion
10192359162,09,17871,163.45848850075987,79da9fb7-5e59-45e...,2021-09-05,Enim Condimentum PC,music shops - mus...,a,6.3,Karen Mathis,1603 Holmes Camp ...,VIC,3373,Female,827726,13,13,4232.756277487095,266.66364548168696
10192359162,09,18613,243.95961514139603,d468ac33-43ce-4af...,2021-09-09,Enim Condimentum PC,music shops - mus...,a,6.3,Susan Collins,6813 Justin Wall,VIC,3224,Female,662119,13,13,4232.756277487095,266.66364548168696
10192359162,09,20213,243.2986102589598,1f6b4f29-c908-476...,2021-09-25,Enim Condimentum PC,music shops - mus...,a,6.3,Barbara Ruiz,4472 Humphrey Pra...,VIC,3127,Female,777120,13,13,4232.756277487095,266.66364548168696
10192359162,09,4487,133.12955672490625,bfd4a38e-9113-45d...,2021-09-20,Enim Condimentum PC,music shops - mus...,a,6.3,Monica Ferguson,270 Miller Pine A...,VIC,3691,Female,293008,13,13,4232.756277487095,266.66364548168696
10192359162,09,16195,601.3713687716552,0fcc6a72-9ace-4dc...,2021-09-11,Enim Condimentum PC,music shops - mus...,a,6.3,Ms. Nancy Coleman,0905 Roberts Isla...,NSW,2580,Female,678525,13,13,4232.756277487095,266.66364548168696
10192359162,09,23182,167.12850029577064,5a63299c-f640-43b...,2021-09-07,Enim Condimentum PC,music shops - mus...,a,6.3,Brittney Larson,44240 Schwartz Or...,WA,6921,Female,1365014,13,13,4232.756277487095,266.66364548168696
10192359162,09,4409,384.7348966145766,ea1e9e45-fd58-48f...,2021-09-16,Enim Condimentum PC,music shops - mus...,a,6.3,Alicia Roberts,22165 Newman Moto...,SA,5076,Female,381525,13,13,4232.756277487095,266.66364548168696
10192359162,09,8396,320.9658915661023,c18f563b-120f-471...,2021-09-09,Enim Condimentum PC,music shops - mus...,a,6.3,Louis Boyd,49462 Mark Forges,NSW,2002,Undisclosed,440460,13,13,4232.756277487095,266.66364548168696
10192359162,09,20308,631.1594496545289,2b7d3779-2689-4be...,2021-09-25,Enim Condimentum PC,music shops - mus...,a,6.3,Cynthia Stone,14027 Morris Orch...,VIC,3860,Female,1494728,13,13,4232.756277487095,266.66364548168696
10192359162,09,19751,39.83130664417192,1c334117-ee10-459...,2021-09-15,Enim Condimentum PC,music shops - mus...,a,6.3,Anthony Camacho,3688 John Forge,SA,5725,Male,780757,13,13,4232.756277487095,266.66364548168696


In [52]:
# using count() to convert back to dataframe, instead of a grouped object
# groupby merchant and month.
# get monthly revenue, monthly order numbers, monthly distinct customers

count_transaction_20220228_20220828_sdf = transaction_20220228_20220828_sdf \
                                                .groupBy("merchant_abn", "month") \
                                                .agg(F.countDistinct("consumer_id"),
                                                     F.count("merchant_abn"),
                                                     F.sum("dollar_value")
                                                    )
aggregate_transaction_20220228_20220828_sdf = transaction_20220228_20220828_sdf \
                                                .join(count_transaction_20220228_20220828_sdf, 
                                                       ["merchant_abn", "month"]) \
                                                .drop_duplicates()
# get monthly take portion
aggregate_transaction_20220228_20220828_sdf = aggregate_transaction_20220228_20220828_sdf \
                                                .withColumn("take_rate",
                                                F.col("take_rate").cast('double'))
aggregate_transaction_20220228_20220828_sdf = aggregate_transaction_20220228_20220828_sdf \
                                                .withColumn("take_portion",
                                                (F.col("take_rate") / 100) * (F.col("sum(dollar_value)")))

aggregate_transaction_20220228_20220828_sdf = aggregate_transaction_20220228_20220828_sdf\
                                                 .withColumnRenamed("count(consumer_id)", "distinct_monthly_consumer_count") \
                                                 .withColumnRenamed("count(merchant_abn)", "monthly_order_count") \
                                                 .withColumnRenamed("sum(dollar_value)", "monthly_revenue")
aggregate_transaction_20220228_20220828_sdf

merchant_abn,month,user_id,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id,distinct_monthly_consumer_count,monthly_order_count,monthly_revenue,take_portion
10023283211,03,201,349.3329692069,4ff47303-5b2d-4f0...,2022-03-03,Felis Limited,"furniture, home f...",e,0.1,Kristen Cordova,4512 Robles Dale ...,VIC,3822,Female,511108,149,150,32456.534424828158,32.456534424828156
10023283211,03,343,105.5363273968829,4d42faba-87ed-49c...,2022-03-17,Felis Limited,"furniture, home f...",e,0.1,Kyle House,27484 Mitchell Isle,NSW,2777,Male,1280031,149,150,32456.534424828158,32.456534424828156
10023283211,03,1991,412.1978189460957,d8ce948f-661c-4b6...,2022-03-24,Felis Limited,"furniture, home f...",e,0.1,Rachel Hampton,348 Adam Knolls,VIC,3596,Female,1129756,149,150,32456.534424828158,32.456534424828156
10023283211,03,3671,143.30928991031547,6effe5a8-4081-4c1...,2022-03-11,Felis Limited,"furniture, home f...",e,0.1,Lori Brown,914 Katrina Glen,VIC,3946,Female,1160719,149,150,32456.534424828158,32.456534424828156
10023283211,03,3808,42.44493820892573,76647936-b01e-4d1...,2022-03-02,Felis Limited,"furniture, home f...",e,0.1,Paul White MD,491 Bird Heights,NSW,2217,Undisclosed,839816,149,150,32456.534424828158,32.456534424828156
10023283211,03,4368,368.14870451598415,62cba1ae-a10f-4e4...,2022-03-20,Felis Limited,"furniture, home f...",e,0.1,Heidi Daniels,517 Padilla Alley,NSW,2568,Female,587808,149,150,32456.534424828158,32.456534424828156
10023283211,03,7448,78.11905768996071,44cc5d30-0b08-47f...,2022-03-29,Felis Limited,"furniture, home f...",e,0.1,April Lee,019 Mosley Vista ...,TAS,7030,Female,553188,149,150,32456.534424828158,32.456534424828156
10023283211,03,8048,449.197244571987,29048534-c2e7-4a2...,2022-03-03,Felis Limited,"furniture, home f...",e,0.1,Laura Flores,8334 Emily Spur A...,NSW,2880,Female,1236175,149,150,32456.534424828158,32.456534424828156
10023283211,03,10608,34.03204257479698,7dae099d-ba3f-470...,2022-03-14,Felis Limited,"furniture, home f...",e,0.1,Jesse Howard,23711 Johnston St...,NSW,1005,Male,413190,149,150,32456.534424828158,32.456534424828156
10023283211,03,18966,173.7380133865698,44ba6254-ffd8-465...,2022-03-16,Felis Limited,"furniture, home f...",e,0.1,Sarah Freeman,52452 Garcia Mall...,QLD,4004,Female,621676,149,150,32456.534424828158,32.456534424828156
